In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('venda_aluguel.txt', sep='\t')
df.columns = ['acao', 'pessoa1', 'pessoa2', 'parcela', 'objeto', 'tipo_preco', 'rodada', 'jogo']
#'saldo_CD1', 'saldo_CD2', 'saldo_CD3', 'saldo_AT1', 'saldo_AT2', 'saldo_AT3', 'saldo_EmpAgr', 'saldo_EmpFer', 'saldo_EmpMaq', 'saldo_EmpSem', 'saldo_FiscalAT', 'saldo_FiscalCD', 'saldo_PrefeitoAT', 'saldo_PrefeitoCD', 'poluicao_mundial', 'selo', 'prox_taxa', 'val_taxa', 'tratamento'

In [3]:
#Inclui colunas para normalizar o banco
serie = pd.Series([])
df['saldo_CD1'] = serie
df['saldo_CD2'] = serie
df['saldo_CD3'] = serie
df['saldo_AT1'] = serie
df['saldo_AT2'] = serie
df['saldo_AT3'] = serie
df['saldo_EmpAgr'] = serie
df['saldo_EmpFer'] = serie
df['saldo_EmpMaq'] = serie
df['saldo_EmpSem'] = serie
df['saldo_FiscalAT'] = serie
df['saldo_FiscalCD'] = serie
df['saldo_PrefeitoAT'] = serie
df['saldo_PrefeitoCD'] = serie
df['poluicao_mundial'] = serie
df['selo'] = serie
df['prox_taxa'] = serie
df['val_taxa'] = serie
df['tratamento'] = serie

In [4]:
#Carrego o restante dos dados
df_acoes = pd.read_csv('acao_ambiental.txt', sep='\t')
df_imposto = pd.read_csv('imposto.txt', sep='\t')
df_multa = pd.read_csv('multa.txt', sep='\t')
df_recebe_selo = pd.read_csv('recebe_selo.txt', sep='\t')
df_transferencia = pd.read_csv('transferencia.txt', sep='\t')
df_troca_taxa = pd.read_csv('troca_taxa.txt', sep='\t')

In [5]:
#Separar venda do aluguel

df['acao'] = df['pessoa1'].apply(lambda x: 'aluguel' if x == 'EmpMaq' else'venda')

In [6]:
df = pd.concat([df, df_acoes, df_imposto, df_multa, df_recebe_selo, df_transferencia, df_troca_taxa])

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [8]:
df.drop(['valor'], axis=1)

,acao,jogo,objeto,parcela,pessoa1,pessoa2,poluicao_mundial,prox_taxa,rodada,saldo_AT1,...,saldo_FiscalAT,saldo_FiscalCD,saldo_PrefeitoAT,saldo_PrefeitoCD,selo,tipo_preco,tratamento,val_taxa,valor2,valor3
0,venda,4,Soja,1.0,EmpSem,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN
1,venda,4,Super premium,1.0,EmpFer,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN
2,aluguel,4,Maquinas 3,1.0,EmpMaq,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN
3,venda,4,Soja,2.0,EmpSem,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN
4,venda,4,Super premium,2.0,EmpFer,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,troca taxa,3,NaN,NaN,CD2,NaN,NaN,2.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0%,NaN,NaN
22,troca taxa,3,NaN,NaN,CD2,NaN,NaN,3.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0%,NaN,NaN
23,troca taxa,2,NaN,NaN,AT2,NaN,NaN,1.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
24,troca taxa,2,NaN,NaN,AT2,NaN,NaN,3.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.0%,NaN,NaN


In [21]:
rodadas = []
valores = [3, 5, 4]

In [25]:
a = 0
for i in range(2,5):
    for y in range(1, (valores[a]+1)):
        rodadas.append(df[(df['jogo'] == i)][df[(df['jogo']== i)]['rodada'] == y])
    a+=1

In [35]:
df_final = pd.DataFrame([])
for i in range(0,12):
    df_final = pd.concat([df_final, rodadas[i]])

In [36]:
df_final

,acao,jogo,objeto,parcela,pessoa1,pessoa2,poluicao_mundial,prox_taxa,rodada,saldo_AT1,...,saldo_FiscalCD,saldo_PrefeitoAT,saldo_PrefeitoCD,selo,tipo_preco,tratamento,val_taxa,valor,valor2,valor3
845,venda,2,Soja,1.0,EmpSem,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN
846,venda,2,Super premium,1.0,EmpAgr,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN
847,venda,2,Soja,2.0,EmpSem,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN
848,venda,2,Super premium,2.0,EmpAgr,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN
849,venda,2,Soja,3.0,EmpSem,AT1,NaN,NaN,1,NaN,...,NaN,NaN,NaN,NaN,baixo,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,imposto,4,NaN,NaN,prefeito EmpSem,AT3,NaN,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600.0,NaN,NaN
39,imposto,4,NaN,NaN,prefeito CD1,CD3,NaN,NaN,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,157.5,NaN,NaN
6,troca taxa,4,NaN,NaN,EmpSem,NaN,NaN,2.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0%,NaN,NaN,NaN
7,troca taxa,4,NaN,NaN,EmpSem,NaN,NaN,3.0,4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,35.0%,NaN,NaN,NaN
